In [3]:
import org.apache.spark.eventhubs.{EventHubsConf, EventPosition}

val connectionString = "" // TODO

val eventHubsConf = EventHubsConf(connectionString)
  .setStartingPosition(EventPosition.fromStartOfStream)
  .setMaxEventsPerTrigger(10)

StatementMeta(sparkpool, 11, 3, Finished, Available)

import org.apache.spark.eventhubs.{EventHubsConf, EventPosition}
connectionString: String = Endpoint=XXX;EntityPath=XXX
eventHubsConf: org.apache.spark.eventhubs.EventHubsConf = org.apache.spark.eventhubs.EventHubsConf@4ea0ad05


In [4]:
spark.conf.set("spark.sql.shuffle.partitions", sc.defaultParallelism)

val eventStreamDF = spark.readStream
  .format("eventhubs")
  .options(eventHubsConf.toMap)
  .load()

eventStreamDF.printSchema()

StatementMeta(sparkpool, 11, 4, Finished, Available)

root
 |-- body: binary (nullable = true)
 |-- partition: string (nullable = true)
 |-- offset: string (nullable = true)
 |-- sequenceNumber: long (nullable = true)
 |-- enqueuedTime: timestamp (nullable = true)
 |-- publisher: string (nullable = true)
 |-- partitionKey: string (nullable = true)
 |-- properties: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- systemProperties: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)

eventStreamDF: org.apache.spark.sql.DataFrame = [body: binary, partition: string ... 7 more fields]


In [5]:
val bodyDF = eventStreamDF.select('body.cast("STRING"))

StatementMeta(sparkpool, 11, 5, Finished, Available)

bodyDF: org.apache.spark.sql.DataFrame = [body: string]


In [8]:
for (s <- spark.streams.active if s.name == "bodyDF") s.stop()

StatementMeta(sparkpool, 11, 8, Finished, Available)

In [9]:
import org.apache.spark.sql.types.{StructField, StructType, StringType, LongType, DoubleType, TimestampType}

lazy val schema = StructType(List(
  StructField("deviceId", StringType),
  StructField("temperature", DoubleType),
  StructField("ts", TimestampType),
  StructField("source", StringType)))

StatementMeta(sparkpool, 11, 9, Finished, Available)

import org.apache.spark.sql.types.{StructField, StructType, StringType, LongType, DoubleType, TimestampType}
schema: org.apache.spark.sql.types.StructType = <lazy>


In [10]:
import org.apache.spark.sql.functions.from_json

val parsedEventsDF = bodyDF.select(
  from_json('body, schema).alias("json"))

parsedEventsDF.printSchema()

StatementMeta(sparkpool, 11, 10, Finished, Available)

root
 |-- json: struct (nullable = true)
 |    |-- deviceId: string (nullable = true)
 |    |-- temperature: double (nullable = true)
 |    |-- ts: timestamp (nullable = true)
 |    |-- source: string (nullable = true)

import org.apache.spark.sql.functions.from_json
parsedEventsDF: org.apache.spark.sql.DataFrame = [json: struct<deviceId: string, temperature: double ... 2 more fields>]


In [12]:
parsedEventsDF

StatementMeta(sparkpool, 11, 12, Finished, Available)

res12: org.apache.spark.sql.DataFrame = [json: struct<deviceId: string, temperature: double ... 2 more fields>]


In [15]:
import org.apache.spark.sql.functions.{from_unixtime, col}

val flatSchemaDF = parsedEventsDF
  .select(
    col("json.deviceId").alias("deviceId"),
    col("json.temperature").alias("temperature"),
    col("json.ts").alias("ts").cast("timestamp"),
    col("json.source").alias("source")
  )

StatementMeta(sparkpool, 11, 15, Finished, Available)

import org.apache.spark.sql.functions.{from_unixtime, col}
flatSchemaDF: org.apache.spark.sql.DataFrame = [deviceId: string, temperature: double ... 2 more fields]


In [18]:
mssparkutils.fs.ls("abfss://deltalake@adbadlsbg.dfs.core.windows.net/")

StatementMeta(sparkpool, 11, 18, Finished, Available)

res15: Array[com.microsoft.spark.notebook.msutils.MSFileInfo] = Array()


In [26]:
val checkpointPath = "abfss://deltalake@adbadlsbg.dfs.core.windows.net/checkpoints"
val rawStreamPath = "abfss://deltalake@adbadlsbg.dfs.core.windows.net/raw"

eventStreamDF
  .withColumn("Body", $"body".cast(StringType))
  .select("Body")
  .writeStream // # Get the DataStreamWriter
  .format("delta") // # Specify the sink type
  .option("checkpointLocation", checkpointPath) // Specify the location of checkpoint files & W-A logs
  .outputMode("append") // Write only new data to the "file"
  .start(rawStreamPath) // Start the job, writing to the specified directory

StatementMeta(sparkpool, 11, 26, Finished, Available)

checkpointPath: String = abfss://deltalake@adbadlsbg.dfs.core.windows.net/checkpoints
rawStreamPath: String = abfss://deltalake@adbadlsbg.dfs.core.windows.net/raw
res21: org.apache.spark.sql.streaming.StreamingQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@3cd35aef


In [20]:
val streaming = flatSchemaDF
  .writeStream
  .format("delta")
  .option("checkpointLocation", checkpointPath + "/raw")
  .outputMode("append")
  .start(rawStreamPath)

StatementMeta(sparkpool, 11, 20, Finished, Available)

streaming: org.apache.spark.sql.streaming.StreamingQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@7a559cdc


In [21]:
streaming.recentProgress

StatementMeta(sparkpool, 11, 21, Finished, Available)

res17: Array[org.apache.spark.sql.streaming.StreamingQueryProgress] =
Array({
  "id" : "1c09c997-9498-4c3d-8dad-173adc12edaf",
  "runId" : "db75cb0a-99fb-4a48-818b-ddaa788e5e19",
  "name" : null,
  "timestamp" : "2022-04-08T09:20:05.744Z",
  "batchId" : 1,
  "numInputRows" : 0,
  "inputRowsPerSecond" : 0.0,
  "processedRowsPerSecond" : 0.0,
  "durationMs" : {
    "getOffset" : 7,
    "triggerExecution" : 352
  },
  "stateOperators" : [ ],
  "sources" : [ {
    "description" : "org.apache.spark.sql.eventhubs.EventHubsSource@7ee42484",
    "startOffset" : {
      "temp" : {
        "0" : 425140
      }
    },
    "endOffset" : {
      "temp" : {
        "0" : 425140
      }
    },
    "numInputRows" : 0,
    "inputRowsPerSecond" : 0.0,
    "processedRowsPerSecond" : 0.0
  } ],
  "sink" : ...


In [27]:
for (s <- spark.streams.active)
  println(s)

StatementMeta(sparkpool, 11, 27, Finished, Available)

org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@3cd35aef


In [29]:
import org.apache.spark.sql.types._

val inputPath = "abfss://deltalake@adbadlsbg.dfs.core.windows.net/raw"

val staticInputDF = 
  spark
    .read
    .format("delta")
    .load(inputPath)

display(staticInputDF)

StatementMeta(sparkpool, 11, 29, Finished, Available)

SynapseWidget(Synapse.DataFrame, 8914f7ad-5bcc-4140-aa24-d74ee0f5a582)


import org.apache.spark.sql.types._
inputPath: String = abfss://deltalake@adbadlsbg.dfs.core.windows.net/raw
staticInputDF: org.apache.spark.sql.DataFrame = [Body: string]
